<a href="https://colab.research.google.com/github/DavoodSZ1993/RL/blob/main/09_DNQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Improt Dependencies

In [11]:
import random
import gym
import numpy as np

from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import os

### Set Parameters

In [23]:
env = gym.make('CartPole-v0')

batch_size = 32
n_episodes = 1001

output_dir = './cartpole'

state_size = env.observation_space.shape[0]
action_size = env.action_space.n

state_size, action_size

(4, 2)

In [ ]:
if not os.path.exists(output_dir):
  os.makedirs(output_dir)